In [32]:
import tensorflow as tf
tf.test.gpu_device_name()
from IPython.core.magic import register_cell_magic
@register_cell_magic
def run_and_save(line, cell):
    'Run and save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)
    code = compile(cell, line, 'exec')
    exec(code, globals())

!nvidia-smi
!pip install mne
!pip install tensorflow-addons
!pip install scikit-learn
!pip install tqdm
!pip install tabulate

import os
try:
    networkname="../results/test"
    os.makedirs(networkname)
except:
    print('Exists')

Wed Mar  3 17:58:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 29%   23C    P8    15W / 250W |  10904MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
%%run_and_save {networkname}/config.py

import matplotlib
from libs import GeneratorCNN
matplotlib.interactive(False)
from importlib import reload
import gc
from libs import model_zoo
import BaseLineModel as approach
import numpy as np
from importlib import reload
from libs import utils
from libs.utils import Bunch
from libs import Multi_Class_Metrics as mcm
from tensorflow.keras import callbacks

reload(approach)
reload(utils)
reload(GeneratorCNN)
reload(model_zoo)
network = model_zoo.ConnectivityModel#EEGNetLawhern#ConnectivityModel#SpatialTemporalMultiClass#ConnectivityModel#Channel_Decision_Small#Channel_Decision_Model#ShallowConvNetMultiClass#SpatialTemporalMultiClass#ShallowConvNetMultiClassLinear##SpatialAverageModel# ConnectivityModel#
gc.collect()
use_stable=True
subjects = {'dementia':[],'control':[],'mci':[]}
utils.get_subjects_wrt_site(subjects,30,30,'dementia')
utils.get_subjects_wrt_site(subjects,100,100,'control')
utils.get_subjects_wrt_site(subjects,1,15,'mci')
subjects['dementia'].remove(27)

use_stable='all'
if use_stable=='order0':
    channel_matches = np.load('pareto_opt_matches.npy')
    channel_matches = utils.get_stable_channels(channel_matches,0).T#[:,44:46]
elif use_stable=='all':
    import json
    with open('A_B_bestpositional_hungarian.json') as f:
        match = json.load(f)
    match['matching0'].pop('info')
    channel_matches = np.array([list(match['matching0'].keys()),list(match['matching0'].values())]).astype(int)
elif use_stable is None:
    channel_matches = np.stack([np.arange(160),np.arange(160)])

site_as_label = False
if site_as_label:
    num_classes = 2
else:
    num_classes = 3


sensitivity_mci_dem = mcm.MultiClassRecall(num_classes=num_classes,
                                           pos_ind=[1,2],
                                           average='macro',
                                           name="mci_dem_sensitivity")
specificity_mci_dem = mcm.MultiClassSpecificity(num_classes=num_classes,
                                                pos_ind=[1,2],
                                                average='macro',
                                                name="mci_dem_Specificity")
f1_mci_dem = mcm.MultiClassF1(num_classes=num_classes,
                              pos_ind=[1,2],average='macro',
                              name="mci_dem_F1")
metrics = ['acc', sensitivity_mci_dem, specificity_mci_dem,f1_mci_dem]


callbacks = [callbacks.EarlyStopping(patience=5,
                                     restore_best_weights=True,
                                     monitor="val_mci_dem_F1",
                                     mode='max',
                                     verbose=1)]
frame_size = utils.Bunch(value=2,unit='s')
fs = 256
network_params = {
    'batch_size':8,
    'use_bn':True,
    'do_ratio':0.5,
    'numTrainEpochs':1,
    'optimizer':'adam',
    'frame_size':frame_size,
    'cross_subjects':True,
    'num_classes':num_classes,
    'n_folds':5,
    'use_class_weights':True,
    'workers':8,
    'multiprocessing':False,
    'monitor':"val_mci_dem_F1",
    'aggregation':'max',
    }

data_params={
             'standardize':'em_astd',#'ema',#'look_at_time',# look_at_time
             'fs':fs,
             'channel_matches':channel_matches,
             'subjects':subjects,
             'test_ratio':0.2,
             'readSubjects_params':{
                 'site_as_label':site_as_label,
                 'l_freq':1,
                 'h_freq':14,
                 'frame_length':utils.time2samples(time_container=frame_size,fs=fs),
                 'bad_samples_path':r'../cache/bad_samples'
                 },
            }
# the first 80 magnetic poles contain the inner magnetic field
SA  = np.load('SA.npy')[:,:80]
SB  = np.load('SB.npy')[:,:80]
pSA = np.load('pSA.npy')[:80]
pSB = np.load('pSB.npy')[:80]

# Build augmentation pipeline for Generator
function_params = {}
function_params['draw_random_time_frame']={
   'frame_length':utils.time2samples(fs=data_params['fs'],time_container=network_params['frame_size']),
   'trial_length':5*60*data_params['fs']}
function_params['additive_correlated_noise']={'sigma_noise':5e-2}
function_params['transform_recording_site']={'sensor_inv':[pSA,pSB],
                                             'sensor_fwd':[SA,SB],
                                             'map_to':'random'}
function_params['apply_channel_matches']={'matches':data_params['channel_matches']}

functions=[GeneratorCNN.draw_random_time_frame,
           GeneratorCNN.additive_correlated_noise,
           GeneratorCNN.transform_recording_site,
           GeneratorCNN.apply_channel_matches
           ]

prob = {'apply_channel_matches':1.0,
        'draw_random_time_frame':1.0,
        'transform_recording_site':1.0,
        'additive_correlated_noise':0.5,
        }

train_aug_pipe=GeneratorCNN.Augment_Pipeline(functions=functions,
                                  params=function_params,
                                  p=prob)
func = [
        GeneratorCNN.draw_random_time_frame,
        GeneratorCNN.transform_recording_site,
        GeneratorCNN.apply_channel_matches]
p = {'transform_recording_site':1,'apply_channel_matches':1,'draw_random_time_frame':1}
aug_params = {
    'transform_recording_site':
       {'map_to':'identity',
        'sensor_inv':[pSA,pSB],
        'sensor_fwd':[SA,SB]},
    'apply_channel_matches':
        {'matches':data_params['channel_matches']},
    'draw_random_time_frame':
        {'frame_length':utils.time2samples(fs=data_params['fs'],time_container=network_params['frame_size']),
        'trial_length':5*60*data_params['fs']}
        }
valid_pipe = GeneratorCNN.Augment_Pipeline(func,aug_params,p)

def get_test_objects(fold,network):
    overlap = 256
    func = [
            GeneratorCNN.draw_continuous_time_frame,
            GeneratorCNN.transform_recording_site,
            GeneratorCNN.apply_channel_matches]
    p = {'transform_recording_site':1,'apply_channel_matches':1,'draw_continuous_time_frame':1}
    aug_params = {
        'transform_recording_site':
           {'map_to':'identity',
            'sensor_inv':[pSA,pSB],
            'sensor_fwd':[SA,SB]},
        'apply_channel_matches':
            {'matches':data_params['channel_matches']},
        'draw_continuous_time_frame':
            {'frame_length':utils.time2samples(fs=data_params['fs'],time_container=network_params['frame_size']),
            #'trial_length':5*60*data_params['fs']
             'overlap':overlap
             }
            }
    test_pipe = GeneratorCNN.Augment_Pipeline(func,aug_params,p)
    test_pipe.params['overlap']=overlap
    input_shape = (512,160,1)
    model = network(input_shape,network_params)
    model = trainer.load_model(fold,model=model)
    from tensorflow.keras import models
    feature_model = models.Model(model.input,model.layers[-3].output)
    return test_pipe, model, feature_model

r
calc scores updata
r


/tf/TeleBrain/Biomag21/libs/utils.py:382: UserWarning: Method is deprecated us 'get_site'.
  warnings.warn("Method is deprecated us 'get_site'.")


reload(mcm)
reload(GeneratorCNN)
reload(approach.GeneratorCNN)
reload(utils)
reload(model_zoo)
reload(approach)
reload(approach.readSubjects)

In [33]:
reload(approach)

trainer = approach.RunTraining(dataDir=r'../cache/raw',
                               train_path_info='./info',
                               static_data_params=data_params,
                               network_params=network_params,
                               completeDir=networkname)

In [19]:
last_model = trainer.start_main(network=network,
                                train_augment_pipeline=train_aug_pipe,
                                valid_augment_pipeline=valid_pipe,
                                metrics=metrics,
                                callbacks_list=callbacks)

Function 'start_main' was already run in this folder.                 Either delete this folder or use 'continue_training'


In [8]:

for fold in np.arange(0,5,1):
    model = trainer.continue_training(fold,network,metrics=metrics,
                                           callbacks_list=callbacks)
    del model
    test_pipe, model, feature_model = get_test_objects(fold,network)
    results = trainer.test_model(last_model,test_pipe,fold=fold,keep_data=False, 
                                 data_loaded=False,iterative=False)
    conf_mat = trainer.evaluate3classes(results[0],results[1],fold=fold,metrics=metrics[1:])
    """
    results_features_test = trainer.test_model(feature_model,test_pipe,fold,
                                                 keep_data=False,data_loaded=True,
                                                 subject_group='test_subjects.json',
                                                 iterative=False,
                                                 make_classification=False)
    results_features_train = trainer.test_model(feature_model,test_pipe,fold,
                                                 keep_data=False,data_loaded=False,
                                                 subject_group='train_subjects.json',
                                                iterative=True,
                                                make_classification=False)
    """

{'mci': 12, 'dementia': 22, 'control': 79}


  0%|          | 0/113 [00:00<?, ?it/s]

100%|██████████| 113/113 [01:20<00:00,  1.41it/s]


Network has shape:  (512, 160, 1)
2
Tracing
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 160, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 512, 160, 16)      576       
_________________________________________________________________
average_pooling2d (AveragePo (None, 256, 160, 16)      0         
_________________________________________________________________
activation (Activation)      (None, 256, 160, 16)      0         
_________________________________________________________________
pairwise_channel_add (Pairwi (None, 256, 159, 16)      0         
_________________________________________________________________
spatial_conv (Conv2D)        (None, 256, 1, 16)        40720     
_________________________________________________________________
batch_normalizati

/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[0 1 2], y=[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1
 1 1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Exponential moving z score with alpha: 0.0005
Train Patients: 
 ['control100' 'control18' 'control19' 'control20' 'control23' 'control26'
 'control27' 'control28' 'control30' 'control31' 'control32' 'control33'
 'control34' 'control35' 'control36' 'control37' 'control38' 'control39'
 'control40' 'control42' 'control43' 'control44' 'control45' 'control46'
 'control47' 'control48' 'control49' 'control50' 'control51' 'control52'
 'control54' 'control55' 'control56' 'control57' 'control58' 'control59'
 'control60' 'control61' 'control62' 'control63' 'control64' 'control65'
 'control66' 'control67' 'control69' 'control70' 'control71' 'control72'
 'control73' 'control74' 'control75' 'control76' 'control77' 'control78'
 'control79' 'control80' 'control81' 'control82' 'control83' 'control85'
 'control86' 'control87' 'control88' 'control90' 'control91' 'control92'
 'control93' 'control94' 'control97' 'control98' 'control99' 'dementia12'
 'dementia13' 'dementia14' 'dementia15' 'dementia17' 'deme

/tf/TeleBrain/Biomag21/libs/GeneratorCNN.py:489: UserWarning: Make sure that the recording site is provided for each subject.
  warnings.warn('Make sure that the recording site is provided for each subject.')


2
2
1893/1893 [==============================] - ETA: 0s - loss: 0.8510 - acc: 0.5927 - mci_dem_sensitivity: 0.6143 - mci_dem_Specificity: 0.9189 - mci_dem_F1: 0.42782
2
2
2
2
2
2
2
INFO:tensorflow:Unsupported signature for serialization: (([TensorSpec(shape=(None, 256, 1, 16), dtype=tf.float32, name='X_split/0'), TensorSpec(shape=(None, 256, 1, 16), dtype=tf.float32, name='X_split/1'), TensorSpec(shape=(None, 256, 1, 16), dtype=tf.float32, name='X_split/2'), TensorSpec(shape=(None, 256, 1, 16), dtype=tf.float32, name='X_split/3'), TensorSpec(shape=(None, 256, 1, 16), dtype=tf.float32, name='X_split/4'), TensorSpec(shape=(None, 256, 1, 16), dtype=tf.float32, name='X_split/5'), TensorSpec(shape=(None, 256, 1, 16), dtype=tf.float32, name='X_split/6'), TensorSpec(shape=(None, 256, 1, 16), dtype=tf.float32, name='X_split/7'), TensorSpec(shape=(None, 256, 1, 16), dtype=tf.float32, name='X_split/8'), TensorSpec(shape=(None, 256, 1, 16), dtype=tf.float32, name='X_split/9'), TensorSpec(shape=(

NameError: name 'model' is not defined

In [34]:

results = trainer.test_model(last_model,test_pipe,fold=fold,keep_data=False, 
                             data_loaded=False,iterative=False)
conf_mat = trainer.evaluate3classes(results[0],results[1],fold=fold,metrics=metrics[1:])

{'mci': 3, 'dementia': 6, 'control': 21}


  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:21<00:00,  1.43it/s]


Exponential moving z score with alpha: 0.0005


100%|██████████| 30/30 [03:26<00:00,  6.88s/it]


| ids         |   y |   pred | site   |
|:------------|----:|-------:|:-------|
| control001  |   0 |      0 | A      |
| control002  |   0 |      1 | B      |
| control003  |   0 |      0 | A      |
| control004  |   0 |      0 | A      |
| control005  |   0 |      0 | A      |
| control006  |   0 |      0 | A      |
| control007  |   0 |      0 | A      |
| control008  |   0 |      1 | B      |
| control009  |   0 |      1 | B      |
| control010  |   0 |      1 | B      |
| control011  |   0 |      0 | A      |
| control012  |   0 |      0 | A      |
| control013  |   0 |      0 | A      |
| control014  |   0 |      1 | B      |
| control015  |   0 |      0 | A      |
| control016  |   0 |      0 | A      |
| control017  |   0 |      1 | B      |
| control021  |   0 |      0 | A      |
| control022  |   0 |      0 | A      |
| control024  |   0 |      0 | A      |
| control025  |   0 |      0 | A      |
| dementia001 |   2 |      0 | A      |
| dementia002 |   2 |      1 | B      |
